In [21]:
import cv2
import numpy as np
from PIL import ImageGrab
import time
import pytesseract
import pygetwindow as gw

# Configure o caminho do Tesseract no seu sistema
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Altere para o caminho do Tesseract no seu sistema

# Nome da janela que você deseja capturar
window_name = "GrandChase v.1.57.09 x64"

# Função para obter a posição da janela
def get_window_area(window_name):
    windows = gw.getWindowsWithTitle(window_name)
    if windows:
        win = windows[0]
        return (win.left, win.top, win.right, win.bottom)
    else:
        print(f"Janela '{window_name}' não encontrada.")
        return None

# Função para capturar a área da janela
def capture_window(area):
    if area:
        screen = ImageGrab.grab(bbox=area)
        screen_np = np.array(screen)
        screen_np = cv2.cvtColor(screen_np, cv2.COLOR_RGB2BGR)
        return screen_np
    return None

# Função para detectar o texto verde (nickname) na imagem
def detect_green_text_and_draw_hitbox(screen_np, name="Rebbids"):
    # Definir uma região de interesse (ROI) na parte inferior da tela, onde o nome geralmente aparece
    roi = screen_np[int(screen_np.shape[0] * 0.7):, :]  # Foco na parte inferior da tela

    # Converter a imagem para o espaço de cores HSV
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Definir os limites para o tom de verde (ajustável conforme necessário)
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])

    # Criar uma máscara que isola apenas os tons de verde
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Aplicar a máscara à imagem original
    result = cv2.bitwise_and(roi, roi, mask=mask)

    # Converter a imagem resultante para escala de cinza para o OCR
    gray_result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

    # Exibir a imagem filtrada (com o texto verde isolado) para visualização
    cv2.imshow("Texto Verde Isolado", result)

    # Aplicar OCR para detectar o texto na imagem filtrada
    detected_text = pytesseract.image_to_data(gray_result, output_type=pytesseract.Output.DICT)

    # Procurar pelo nome especificado na imagem
    for i, text in enumerate(detected_text['text']):
        if name.lower() in text.lower():  # Verificar se o nome está presente
            x = detected_text['left'][i]
            y = detected_text['top'][i]
            w = detected_text['width'][i]
            h = detected_text['height'][i]

            # Ajustar as coordenadas de volta para a imagem original
            y += int(screen_np.shape[0] * 0.7)

            # Aumentar a largura do retângulo
            extra_width = 55  # Ajuste este valor para deixar o retângulo mais largo
            x -= extra_width // 2  # Subtrai uma parte para a esquerda
            w += extra_width  # Adiciona o mesmo valor para a largura total

            # Desenhar um retângulo em torno do nome detectado
            cv2.rectangle(screen_np, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Desenhar uma hitbox mais acima do nome (presumindo que o personagem está logo acima do nome)
            hitbox_height = 230  # Ajuste conforme necessário
            cv2.rectangle(screen_np, (x, y - hitbox_height), (x + w, y), (0, 0, 255), 2)
            break

    # Exibir a imagem com a hitbox
    cv2.imshow("Hitbox", screen_np)

# Loop principal para capturar a janela e desenhar a hitbox
while True:
    # Obter a área da janela do jogo
    window_area = get_window_area(window_name)

    if window_area:
        # Capturar a janela
        screen = capture_window(window_area)

        if screen is not None:
            # Detectar o texto verde "Rebbids" e desenhar a hitbox
            detect_green_text_and_draw_hitbox(screen, name="Rebbids")

        # Fechar a janela ao pressionar 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    time.sleep(0.1)

# Fechar todas as janelas do OpenCV
cv2.destroyAllWindows()
